# GAN Evaluation - Frechet Inception Distance (FID) Score

## Basic [Concept](https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/)

In [1]:
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random

from scipy.linalg import sqrtm

In [2]:
def calculate_fid_score(fvector1, fvector2):
    # calculate mean and covariance statistics
    mean1, sigma1 = fvector1.mean(axis=0), cov(fvector1, rowvar=False)
    mean2, sigma2 = fvector2.mean(axis=0), cov(fvector2, rowvar=False)
    
    # calculate sum squared difference between two mean vectors
    ssdiff = np.sum((mean1 - mean2) ** 2.0)
    
    # calculate sqrt of product between two covariance metrics
    covmean = sqrtm(sigma1.dot(sigma2))
    
    # some elements in the resulting matrix may be imaginary
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [3]:
# define two collections of activations
fvector1 = random(10*2048)
fvector1 = fvector1.reshape((10,2048))
fvector2 = random(10*2048)
fvector2 = fvector2.reshape((10,2048))

In [4]:
# fid between act1 and act1
fid = calculate_fid_score(fvector1, fvector1)
print('FID (identical feature vectors): %.3f' % fid)

FID (identical feature vectors): -0.000


In [5]:
# fid between act1 and act2
fid = calculate_fid_score(fvector1, fvector2)
print('FID (different feature vectors): %.3f' % fid)

FID (different feature vectors): 356.899


## FID in Pytorch

In [6]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
from torch.utils.data import Dataset

from torchvision.models.inception import inception_v3

In [7]:
def calculate_fid_score(real_images, fake_images, batch_size=8, resize=False, cuda=False):
       
    N_real = len(real_images)
    N_fake = len(fake_images)
    
    assert batch_size > 0
    assert N_real > batch_size
    assert N_fake > batch_size
    
    # set up dtype for inception v3 model
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor
        
    # set up dataloader
    dl_real = torch.utils.data.DataLoader(real_images, batch_size=batch_size)
    dl_fake = torch.utils.data.DataLoader(fake_images, batch_size=batch_size)
    
    # load inception model
    print("Loading Inception V3 model...")
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    
    # exclude top layer (not sure the way to exclude the layer, please figure out later)
    without_top_layer = list(inception_model.children())[:-4]
    inception_model = nn.Sequential(*without_top_layer)
    
    inception_model.eval()
    
    up = nn.Upsample(size=(299, 299), mode='bilinear', align_corners=False).type(dtype)
    def get_feature_vectores(x1, x2):
        if resize:
            x1 = up(x1)
            x2 = up(x2)
        fvector1 = inception_model(x1)
        fvector2 = inception_model(x2)
        return fvector1.data.cpu().numpy(), fvector2.data.cpu().numpy()
    
    # get feature vectores of real and fake images
    print("Getting feature vectores of real and fake images...")
    
    fid_scores = []
    for i, (images) in enumerate(zip(dl_real, dl_fake)):

        # limit images batches to compare with
        if i == 20: break
            
        images_real = images[0].type(dtype)
        images_fake = images[1].type(dtype)
        
        images_realv = Variable(images_real)
        images_fakev = Variable(images_fake)

        fvector1, fvector2 = get_feature_vectores(images_realv, images_fakev)
        
        # calculate mean and covariance statistics
        mean1, sigma1 = fvector1.mean(axis=0), cov(fvector1, rowvar=False)
        mean2, sigma2 = fvector2.mean(axis=0), cov(fvector2, rowvar=False)

        # calculate sum squared difference between two mean vectors
        ssdiff = np.sum((mean1 - mean2) ** 2.0)
        
        # calculate sqrt of product between two covariance metrics
        covmean = sqrtm(sigma1.dot(sigma2))
        
        # some elements in the resulting matrix may be imaginary
        # check and correct imaginary numbers from sqrt
        if iscomplexobj(covmean):
            covmean = covmean.real

        # calculate score
        fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
        fid_scores.append(fid)
        
    return np.mean(fid_scores)

In [8]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

cifar = dset.CIFAR10(root='data/', download=True,
                         transform=transforms.Compose([
                             transforms.Resize(32),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                         ])
)

class IgnoreLabelDataset(torch.utils.data.Dataset):
    def __init__(self, orig):
        self.orig = orig

    def __getitem__(self, index):
        return self.orig[index][0]

    def __len__(self):
        return len(self.orig)

Files already downloaded and verified


In [9]:
print("Calculating Inception Score...")
fid_score = calculate_fid_score(real_images=IgnoreLabelDataset(cifar), 
                                fake_images=IgnoreLabelDataset(cifar), 
                                batch_size=4, resize=True, cuda=True)
print("FID score:",fid_score)

Calculating Inception Score...
Loading Inception V3 model...
Getting feature vectores of real and fake images...
FID score: -0.00028930964644207355


## *Experiments with Inception Model

In [ ]:
from numpy.random import randint

# define two fake collections of images
images1 = randint(0, 255, 10*299*299*3)
images1 = images1.reshape((10,3,299,299))
images1 = torch.from_numpy(images1)
images1 = images1.type(torch.cuda.FloatTensor)
images1 = Variable(images1)

print("Loading Inception V3 model...")
inception_model = inception_v3(pretrained=True, transform_input=False).type(torch.cuda.FloatTensor)

# exclude top layer
without_top_layer = list(inception_model.children())[:-4]
inception_model = nn.Sequential(*without_top_layer)
inception_model.eval()
fvector1 = inception_model(images1)

---